Most methods in this notebook are initially done by David Robinson on his Youtube series Tidy Tuesday screencast with R: https://www.youtube.com/watch?v=AQzZNIyjyWM

This notebook attempts to recreate and improve on that analysis by translating it into Python and adding my own further analysis

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use("ggplot")

In [19]:
wine_rating_df = pd.read_csv('winemag-data-130k-v2.csv')
wine_rating_df.drop(columns='Unnamed: 0', inplace = True)
wine_rating_df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [10]:
wine_rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
Unnamed: 0               129971 non-null int64
country                  129908 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(2), object(11)
memory usage: 13.9+ MB


In [14]:
wine_rating_df.nunique()

Unnamed: 0               129971
country                      43
description              119955
designation               37979
points                       21
price                       390
province                    425
region_1                   1229
region_2                     17
taster_name                  19
taster_twitter_handle        15
title                    118840
variety                     707
winery                    16757
dtype: int64

### Fill in missing countries by matching with existing winery names

The assumption is that each winery only exist in one country (checked with .nunique()). Therefore, Some of the missing 'country' values may be filled in by referring to another data row with a matching 'winery' value.

In [164]:
# List of wineries with one or more missing 'country' values
missing_country = wine_rating_df[wine_rating_df["country"].isnull()]['winery'].unique()
missing_country

array(['Kakhetia Traditional Winemaking', 'Tsililis', 'Ross-idi',
       'Orbelus', 'St. Donat', 'Stone Castle', 'Lismore', 'Somlói Vándor',
       'Les Frères Dutruy', 'El Capricho', 'Amiran Vepkhvadze',
       'Büyülübağ'], dtype=object)

In [165]:
# only wineries with 'country' values inferable from other data rows are chosen
df_matched_country = wine_rating_df[['country', 'winery']][wine_rating_df['winery'].isin(missing_country)]
missing_country_matchable = df_matched_country.dropna()['winery'].unique()
missing_country_matchable

array([], dtype=object)

In [166]:
winery_countery_match_df = wine_rating_df[['country','winery']][(wine_rating_df['winery'].isin(missing_country_matchable)) & (wine_rating_df['country'].notnull())].drop_duplicates(subset = 'winery', keep='first')
winery_countery_match_df.set_index('winery',inplace = True)
winery_countery_match_dict = winery_countery_match_df.to_dict()
for key in winery_countery_match_dict['country']:
    rowidx = wine_rating_df[wine_rating_df['winery'] == key].index
    wine_rating_df.loc[rowidx, 'country'] = winery_countery_match_dict['country'][key]

In [167]:
wine_rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 13 columns):
country                  129940 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB


32 missing 'country' values infered from their 'winery' values